In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

autores_df = pd.read_csv('autores2.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 4.94 s, sys: 274 ms, total: 5.22 s
Wall time: 5.21 s


In [3]:
autores_df.shape

(3054360, 4)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


# Autores con país desconocido

In [5]:
autores_pais_desconocido_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] == 'DESCONOCIDO')
]

In [6]:
autores_pais_desconocido_df.shape

(1682347, 4)

In [7]:
autores_pais_desconocido_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


In [8]:
count_autores_desconocidos_sr = \
    autores_pais_desconocido_df.groupby(['AUTOR'])['TITULO'].count().sort_values(ascending=False)

In [10]:
count_autores_desconocidos_sr.head()

AUTOR
Colombia                                                   11526
Friedrich Wilhelm Nietzsche                                 5876
Ivar Da Coll                                                5350
Keiko Kasza                                                 5336
Instituto Colombiano de Normas Técnicas y Certificación     4333
Name: TITULO, dtype: int64

In [49]:
THRESHOLD = 500

In [50]:
top_autores_desconocidos_sr = count_autores_desconocidos_sr[lambda x: x >= THRESHOLD]

In [51]:
top_autores_desconocidos_sr.shape

(290,)

In [31]:
top_autores_desconocidos_sr

AUTOR
Colombia                                                   11526
Friedrich Wilhelm Nietzsche                                 5876
Ivar Da Coll                                                5350
Keiko Kasza                                                 5336
Instituto Colombiano de Normas Técnicas y Certificación     4333
Fedor Mijailovich Dostoievski                               4296
Graciela Montes                                             3686
Eduardo Galeano                                             2927
Miguel de Cervantes Saavedra                                2746
Aristóteles                                                 2743
James Preller                                               2516
Dennis G Zill                                               2497
Jacob Ludwig Karl Grimm                                     2386
Xavier Deneux                                               2332
Jean Paul Sartre                                            2326
Andrés Caicedo Este

## País autor

In [33]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{ ?author wdt:P106 wd:Q36180 . }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [34]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{ ?author rdf:type dbo:Writer . }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [35]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [36]:
def choose(opt_list):
    return Counter(opt_list).most_common(1)[0][0]

In [37]:
# def get_pais_autor(name):
#     r = 'DESCONOCIDO'
#     opt_dict = {}
#     for lang in ['en', 'es']:
#         opt_dict[lang] = {}
#         for db in ['dbp', 'wdt']:
#             opt_dict[lang][db] = get_pais_autor_data(db, name, lang)
#         opts = opt_dict[lang].values()
#         if all(map(lambda x: x != 'UNKOWN', opts)):
#             r = choose(opts)
#             break
#     if r == 'DESCONOCIDO':
#         for lang in ['en', 'es']:
#             opts = opt_dict[lang].values()
#             if any(map(lambda x: x != 'UNKOWN', opts)):
#                 opts = [opt for opt in opts if opt != 'UNKOWN']
#                 r = choose(opts)
#                 break
#     return r

def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['dbp', 'wdt']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [38]:
%%time

autores_paises_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_paises_dict[x['AUTOR']] = x['PAIS_AUTOR']

CPU times: user 19.7 s, sys: 14.3 ms, total: 19.7 s
Wall time: 19.7 s


In [39]:
def combinaciones(ls):
    r = []
    for x in ls:
        aux = list(ls)
        aux.remove(x)
        r.append(aux)
    return r

In [40]:
for autor in top_autores_desconocidos_sr.index[:4]:
    nombres = autor.split()
    if len(nombres) > 1:
        for c in combinaciones(nombres):
            aux = ' '.join(c)
            pais = get_pais_autor(aux)
            if pais != 'DESCONOCIDO':
                autores_paises_dict[autor] = pais
                break

In [41]:
%%time

def process_pais(x):
    return autores_paises_dict[x['AUTOR']]

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais, axis=1)

Index(['Colombia', 'Friedrich Wilhelm Nietzsche', 'Ivar Da Coll',
       'Keiko Kasza'],
      dtype='object', name='AUTOR')

# Saving results

In [43]:
autores_df.to_csv('autores3.csv', index=False)

'DESCONOCIDO'